In [22]:
import sys
import yaml
import itertools
import subprocess
import multiprocessing

In [23]:
from pathlib import Path

# Compile original Lyra2 implementations

Need to import python functions that compile Lyra from a git submodule. Manipulate path as per https://stackoverflow.com/a/29747054/1269892

In [24]:
sys.path.append(str(Path('../lyra/Lyra2/tests').resolve()))

In [25]:
from harness import build_lyra2, unlist_values
from harness import compose_sponge_name, compose_lyra2_name

In [26]:
with open('harness.yml', 'r') as config:
    params = yaml.load(config)

In [27]:
build_lyra2(params)

In [28]:
build_path0 = Path('./bin42').resolve()
if not build_path0.exists():
    print('Could not find ./bin42, directory for original executables')

# Compile ported Java implementation

In [29]:
subprocess.run(['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'])

CompletedProcess(args=['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'], returncode=0)

In [30]:
build_path1 = Path('./target').resolve()
if not build_path1.exists():
    print('Could not ./target, directory for compiled executables')

# Class to measure time and memory usage

Resources used for memory measurements:

https://stackoverflow.com/questions/22372960/is-this-explanation-about-vss-rss-pss-uss-accurate

In [31]:
import time
import psutil

In [32]:
from statistics import median

In [33]:
class ProcessObserver:
    def __init__(self, ntimes=3, mtimes=3):
        # Number of times to measure elapsed time
        self.ntimes = ntimes
        # Number of times to measure consumed memory
        self.mtimes = mtimes

    def run(self, *args):
        # Run the process self.ntimes and see how long it takes
        times = [-1 for i in range(self.ntimes)]
        for i in range(self.ntimes):
            fst = time.time()
            
            process = subprocess.run(*args)
                        
            lst = time.time()
            
            if process.returncode != 0:
                print(args[0] + ' failed to run')

                continue
            
            times[i] = lst - fst
            
        mtime = median(times)
        
        process = subprocess.Popen(*args)
        
        # Approximate running time of the process is known
        # The process has just been started asynchronously
        # Measure its memory usage (which is complicated)
        pss_mems = [-1 for i in range(self.mtimes)]
        uss_mems = [-1 for i in range(self.mtimes)]
        for i in range(self.mtimes):
            if process.poll() is not None:
                break # the process no longer runs, break

            # the process will finish when you least expect it
            # so wrap everything into try-except and handle it
            try:
                p = psutil.Process(process.pid)
                mem = p.memory_full_info()

                # total private memory + proporional size for the 3pp libraries
                pss_mems[i] = mem.pss
                # total private memory of a process (unique to the process)
                uss_mems[i] = mem.uss
            except: 
                break # the process no longer runs, break

            time.sleep(mtime / self.mtimes)
            
            
        return [median(times), max(pss_mems), max(uss_mems)]

# Perform the measurements

In [34]:
password, salt = 'password', 'salt'

In [35]:
klens =  [1, 10, 100, 1000, 10000]

In [36]:
tcosts = [1, 10, 25, 50, 75, 100]

In [37]:
mcosts = [3, 10, 100, 1000, 10000] # there must be at least 3 rows in the memory matrix

In [38]:
observer = ProcessObserver(ntimes=1, mtimes=5)

In [ ]:
def run_one_configuration(matrix):
    option = matrix['option']

    threads = matrix['threads']
    columns = matrix['columns']

    sponge = matrix['sponge']
    rounds = matrix['rounds']
    blocks = matrix['blocks']

    bench = matrix['bench']

    [sponge, _] = compose_sponge_name(sponge)

    name = compose_lyra2_name(
        option, threads, columns, sponge, rounds, blocks
    )
    
    executable0 = build_path0.joinpath(name)
    # Java implementataion accepts sponge type, number of rounds etc as runtime parameters
    executable1 = build_path1.joinpath('lyra2-1.2-SNAPSHOT-jar-with-dependencies.jar')

    time_data0 = {}
    time_data1 = {}
    
    for klen, tcost, mcost in itertools.product(klens, tcosts, mcosts):
        
        time_data0[klen, tcost, mcost] = observer.run([
            executable0
            , password
            , salt
            , str(klen)
            , str(tcost)
            , str(mcost)
        ])
        
        time_data1[klen, tcost, mcost] = observer.run([
            'java'
            , '-jar'         , str(executable1)
            , '--blocks'     , str(blocks)
            , '--columns'    , str(columns)
            , '--half-rounds', str(rounds)
            , '--sponge'     , sponge
            , password
            , salt
            , str(klen)
            , str(tcost)
            , str(mcost)
        ])
        
        print(name, klen, tcost, mcost)
        print(time_data1[klen, tcost, mcost])
        
    return name, time_data0, time_data1

In [ ]:
measurements0 = {}
measurements1 = {}

with multiprocessing.Pool(4) as pool:
    for name, measurement0, measurement1 in pool.map(run_one_configuration, unlist_values(params['matrix'])):
        measurements0[name] = measurement0
        measurements1[name] = measurement1

lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-1-blocks-8 1 1 3
[0.2035067081451416, 20201472, 16752640]
lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-1-blocks-12 1 1 3
[0.2047107219696045, 18091008, 14639104]
lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-12-blocks-8 1 1 3
[0.21818923950195312, 20742144, 17350656]
lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-12-blocks-12 1 1 3
[0.22814536094665527, 22297600, 17821696]
lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-1-blocks-8 1 1 10
[0.2931973934173584, 20330496, 16859136]
lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-1-blocks-12 1 1 10
[0.25942277908325195, 23014400, 19554304]
lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-12-blocks-8 1 1 10
[0.3033943176269531, 24082432, 17686528]
lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-12-blocks-12 1 1 10
[0.3016843795776367, 24548352, 17928192]
l